# Tech Challenge - Fine-tuning para Produtos Amazon

**Objetivo**: Executar fine-tuning de um foundation model usando o dataset AmazonTitles-1.3MM para gerar descrições de produtos baseadas em títulos.

**Dataset**: Utilizaremos o arquivo `trn.json.gz` que contém títulos e descrições de produtos da Amazon.

**Modelo Escolhido**: Llama 3-8B com Unsloth para otimização de treinamento.

---

## 📋 Índice
1. [Configuração Inicial](#1-configuracao-inicial)
2. [Exploração dos Dados](#2-exploracao-dos-dados)
3. [Preparação do Dataset](#3-preparacao-do-dataset)
4. [Teste do Modelo Base](#4-teste-do-modelo-base)
5. [Fine-tuning](#5-fine-tuning)
6. [Teste do Modelo Treinado](#6-teste-do-modelo-treinado)
7. [Demonstração Interativa](#7-demonstracao-interativa)

---

## 1. Configuração Inicial

### 1.1 Montagem do Google Drive
Primeiro, vamos montar o Google Drive para acessar e salvar nossos arquivos.

In [ ]:
from google.colab import drive
import os

# Monta o Google Drive
drive.mount('/content/drive')

# Define o diretório de trabalho (usando o mesmo diretório onde está o arquivo de dados)
WORK_DIR = '/content/drive/MyDrive/FineTunning/TechChallenge03'
os.makedirs(WORK_DIR, exist_ok=True)

print(f"✅ Google Drive montado com sucesso!")
print(f"📁 Diretório de trabalho: {WORK_DIR}")

# Verifica se o diretório existe e lista os arquivos
if os.path.exists(WORK_DIR):
    files_in_dir = os.listdir(WORK_DIR)
    print(f"📋 Arquivos no diretório: {files_in_dir}")
else:
    print(f"⚠️ Diretório não existe, será criado: {WORK_DIR}")

### 1.2 Instalação das Dependências

Instalamos as bibliotecas necessárias:
- **Unsloth**: Otimização para fine-tuning eficiente
- **Transformers**: Biblioteca principal para modelos de linguagem
- **Datasets**: Para manipulação de datasets
- **TRL**: Para treinamento de modelos de linguagem

In [ ]:
# Instalação das dependências principais
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets torch

print("✅ Todas as dependências foram instaladas com sucesso!")

### 1.3 Importação das Bibliotecas e Configurações Iniciais

In [ ]:
# Imports necessários
import json
import gzip
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
import torch
from transformers import TrainingArguments, TextStreamer
from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso!")
print(f"🔥 CUDA disponível: {torch.cuda.is_available()}")
print(f"💾 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Não disponível'}")

### 1.4 Configurações do Modelo e Treinamento

In [ ]:
# Configurações principais
CONFIG = {
    # Configurações do modelo
    'model_name': "unsloth/llama-3-8b-bnb-4bit",
    'max_seq_length': 2048,
    'dtype': None,  # Será determinado automaticamente
    'load_in_4bit': True,
    
    # Configurações do dataset
    'data_file': '/content/drive/MyDrive/FineTunning/TechChallenge03/trn.json.gz',
    'sample_size': 10000,  # Número de amostras para treinamento (pode ajustar)
    'test_size': 100,  # Número de amostras para teste
    
    # Configurações do fine-tuning
    'lora_r': 16,
    'lora_alpha': 16,
    'lora_dropout': 0,
    'max_steps': 100,  # Ajuste conforme necessário
    'learning_rate': 2e-4,
    'batch_size': 2,
    'gradient_accumulation_steps': 4,
    
    # Caminhos - usando o mesmo diretório base
    'base_dir': '/content/drive/MyDrive/FineTunning/TechChallenge03',
    'output_dir': '/content/drive/MyDrive/FineTunning/TechChallenge03/outputs',
    'model_save_path': '/content/drive/MyDrive/FineTunning/TechChallenge03/amazon_model',
}

# Criação dos diretórios
os.makedirs(CONFIG['base_dir'], exist_ok=True)
os.makedirs(CONFIG['output_dir'], exist_ok=True)
os.makedirs(CONFIG['model_save_path'], exist_ok=True)

print("⚙️ Configurações definidas:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 2. Exploração dos Dados

### 2.1 Upload do Arquivo de Dados

Primeiro, você precisa fazer upload do arquivo `trn.json.gz` para o Colab.
Execute a célula abaixo e faça upload do arquivo quando solicitado.

In [ ]:
import os

# Define o caminho para o arquivo no Google Drive
DATA_FILE_PATH = '/content/drive/MyDrive/FineTunning/TechChallenge03/trn.json.gz'

# Verifica se o arquivo existe
if os.path.exists(DATA_FILE_PATH):
    print("✅ Arquivo trn.json.gz encontrado no Google Drive!")
    print(f"📁 Caminho: {DATA_FILE_PATH}")
    
    # Verifica o tamanho do arquivo
    file_size = os.path.getsize(DATA_FILE_PATH)
    print(f"📊 Tamanho do arquivo: {file_size / (1024*1024):.1f} MB")
else:
    print("❌ Arquivo não encontrado no caminho especificado.")
    print(f"❌ Caminho verificado: {DATA_FILE_PATH}")
    print("💡 Certifique-se de que o arquivo trn.json.gz está no diretório correto do Google Drive.")
    DATA_FILE_PATH = None

### 2.2 Carregamento dos Dados

Vamos carregar o dataset completo (ou conforme configurado) e analisar sua estrutura para entender melhor os dados com que estamos trabalhando.

In [ ]:
def load_amazon_data(file_path, sample_size=None):
    """
    Carrega os dados do arquivo JSON comprimido, extraindo apenas title e content
    
    Args:
        file_path: Caminho para o arquivo trn.json.gz
        sample_size: Número de amostras a carregar (None para carregar tudo)
    
    Returns:
        Lista de dicionários com apenas os campos title e content
    """
    data = []
    
    print(f"📖 Carregando dados de {file_path}...")
    print("🎯 Extraindo apenas os campos 'title' e 'content'")
    
    try:
        with gzip.open(file_path, 'rt', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if sample_size and i >= sample_size:
                    break
                    
                try:
                    json_obj = json.loads(line.strip())
                    
                    # Extrai apenas title e content, desconsiderando outros campos
                    if 'title' in json_obj and 'content' in json_obj:
                        clean_item = {
                            'title': json_obj['title'].strip(),
                            'content': json_obj['content'].strip()
                        }
                        
                        # Só adiciona se ambos os campos não estão vazios
                        if clean_item['title'] and clean_item['content']:
                            data.append(clean_item)
                            
                except json.JSONDecodeError:
                    continue
                    
                # Progress update
                if (i + 1) % 1000 == 0:
                    print(f"  Processadas {i + 1} linhas, válidas: {len(data)}")
    
    except Exception as e:
        print(f"❌ Erro ao carregar dados: {e}")
        return []
    
    print(f"✅ Dados carregados com sucesso!")
    print(f"📊 Total de amostras válidas: {len(data)}")
    print(f"🎯 Campos por amostra: title, content")
    return data

# Carrega o dataset conforme configuração (completo ou amostra)
if DATA_FILE_PATH:
    print(f"🔄 Carregando {CONFIG['sample_size']} amostras conforme configuração...")
    raw_data = load_amazon_data(DATA_FILE_PATH, sample_size=CONFIG['sample_size'])
else:
    print("❌ Arquivo de dados não disponível. Execute a célula de verificação primeiro.")
    raw_data = []

In [ ]:
# Análise da estrutura dos dados brutos (antes da limpeza)
if raw_data:
    print("🔍 ANÁLISE DOS DADOS BRUTOS (ANTES DA LIMPEZA)")
    print("=" * 50)
    
    # Exemplo de uma amostra
    print("📝 Exemplo de uma amostra (apenas title e content):")
    sample_item = raw_data[0]
    for key, value in sample_item.items():
        print(f"  {key}: {value}")
    
    print("\n" + "=" * 50)
    
    # Estatísticas gerais
    print(f"📊 ESTATÍSTICAS GERAIS DOS DADOS BRUTOS:")
    print(f"  Total de amostras carregadas: {len(raw_data)}")
    print(f"  Campos utilizados: title, content")
    print(f"  Outros campos: desconsiderados conforme solicitado")
    
    # Análise de qualidade inicial
    print(f"\n🔍 ANÁLISE DE QUALIDADE INICIAL:")
    
    # Verifica tamanhos dos textos
    title_lengths = [len(item['title']) for item in raw_data]
    content_lengths = [len(item['content']) for item in raw_data]
    
    print(f"  Títulos muito curtos (<3 chars): {sum(1 for x in title_lengths if x < 3)}")
    print(f"  Títulos muito longos (>200 chars): {sum(1 for x in title_lengths if x > 200)}")
    print(f"  Conteúdo muito curto (<5 chars): {sum(1 for x in content_lengths if x < 5)}")
    print(f"  Conteúdo muito longo (>1000 chars): {sum(1 for x in content_lengths if x > 1000)}")
    
    # Verifica duplicatas
    unique_titles = len(set(item['title'].lower() for item in raw_data))
    duplicates = len(raw_data) - unique_titles
    print(f"  Títulos duplicados: {duplicates}")
    
    print(f"\n⚠️ Dados precisam de limpeza antes do treinamento!")
    
else:
    print("❌ Nenhum dado foi carregado.")

In [ ]:
# Análise detalhada dos dados brutos
if raw_data:
    print("📏 ANÁLISE DETALHADA DOS DADOS BRUTOS")
    print("=" * 50)
    
    # Calcula estatísticas de comprimento
    title_lengths = [len(item['title']) for item in raw_data]
    content_lengths = [len(item['content']) for item in raw_data]
    
    title_words = [len(item['title'].split()) for item in raw_data]
    content_words = [len(item['content'].split()) for item in raw_data]
    
    print("📝 Comprimento em caracteres:")
    print(f"  Títulos - Mín: {min(title_lengths)}, Máx: {max(title_lengths)}, Média: {np.mean(title_lengths):.1f}")
    print(f"  Conteúdo - Mín: {min(content_lengths)}, Máx: {max(content_lengths)}, Média: {np.mean(content_lengths):.1f}")
    
    print("\n🔤 Comprimento em palavras:")
    print(f"  Títulos - Mín: {min(title_words)}, Máx: {max(title_words)}, Média: {np.mean(title_words):.1f}")
    print(f"  Conteúdo - Mín: {min(content_words)}, Máx: {max(content_words)}, Média: {np.mean(content_words):.1f}")
    
    # Exemplos de diferentes tamanhos
    print("\n📋 EXEMPLOS DE PRODUTOS (DADOS BRUTOS):")
    print("=" * 50)
    
    # Título mais curto
    shortest_idx = title_lengths.index(min(title_lengths))
    print(f"🔸 Título mais curto ({len(raw_data[shortest_idx]['title'])} chars):")
    print(f"  Título: {raw_data[shortest_idx]['title']}")
    print(f"  Conteúdo: {raw_data[shortest_idx]['content']}")
    
    print("\n" + "-" * 30)
    
    # Título mais longo
    longest_idx = title_lengths.index(max(title_lengths))
    print(f"🔸 Título mais longo ({len(raw_data[longest_idx]['title'])} chars):")
    print(f"  Título: {raw_data[longest_idx]['title']}")
    print(f"  Conteúdo: {raw_data[longest_idx]['content'][:200]}...")
    
    print("\n" + "-" * 30)
    
    # Exemplo aleatório
    import random
    random_idx = random.randint(0, len(raw_data)-1)
    print(f"🔸 Exemplo aleatório:")
    print(f"  Título: {raw_data[random_idx]['title']}")
    print(f"  Conteúdo: {raw_data[random_idx]['content']}")
    
else:
    print("❌ Nenhum dado disponível para análise.")

In [ ]:
# Visualizações dos dados brutos
if raw_data:
    print("📊 CRIANDO VISUALIZAÇÕES DOS DADOS BRUTOS")
    print("=" * 50)
    
    # Recalcula as estatísticas para as visualizações
    title_lengths = [len(item['title']) for item in raw_data]
    content_lengths = [len(item['content']) for item in raw_data]
    title_words = [len(item['title'].split()) for item in raw_data]
    content_words = [len(item['content'].split()) for item in raw_data]
    
    # Configuração do matplotlib
    plt.style.use('default')
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Análise dos Dados Brutos - Amazon Products', fontsize=16, fontweight='bold')
    
    # Gráfico 1: Distribuição do comprimento dos títulos
    axes[0,0].hist(title_lengths, bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[0,0].set_title('Distribuição - Comprimento dos Títulos (caracteres)')
    axes[0,0].set_xlabel('Número de caracteres')
    axes[0,0].set_ylabel('Frequência')
    axes[0,0].grid(True, alpha=0.3)
    axes[0,0].axvline(x=3, color='red', linestyle='--', alpha=0.7, label='Mín (3)')
    axes[0,0].axvline(x=200, color='red', linestyle='--', alpha=0.7, label='Máx (200)')
    axes[0,0].legend()
    
    # Gráfico 2: Distribuição do comprimento do conteúdo
    axes[0,1].hist(content_lengths, bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[0,1].set_title('Distribuição - Comprimento do Conteúdo (caracteres)')
    axes[0,1].set_xlabel('Número de caracteres')
    axes[0,1].set_ylabel('Frequência')
    axes[0,1].grid(True, alpha=0.3)
    axes[0,1].axvline(x=5, color='red', linestyle='--', alpha=0.7, label='Mín (5)')
    axes[0,1].axvline(x=1000, color='red', linestyle='--', alpha=0.7, label='Máx (1000)')
    axes[0,1].legend()
    
    # Gráfico 3: Distribuição de palavras nos títulos
    axes[1,0].hist(title_words, bins=20, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[1,0].set_title('Distribuição - Palavras nos Títulos')
    axes[1,0].set_xlabel('Número de palavras')
    axes[1,0].set_ylabel('Frequência')
    axes[1,0].grid(True, alpha=0.3)
    
    # Gráfico 4: Relação entre título e conteúdo
    axes[1,1].scatter(title_lengths, content_lengths, alpha=0.5, color='purple', s=10)
    axes[1,1].set_title('Relação: Título vs Conteúdo (caracteres)')
    axes[1,1].set_xlabel('Comprimento do título')
    axes[1,1].set_ylabel('Comprimento do conteúdo')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualizações dos dados brutos criadas!")
    print("⚠️ Linhas vermelhas mostram limites para limpeza")
    
else:
    print("❌ Nenhum dado disponível para visualização.")

## 3. Preparação do Dataset

### 3.1 Formatação dos Dados para Fine-tuning

Agora vamos formatar os dados no padrão esperado pelo modelo. Criaremos prompts estruturados onde:
- **Input**: Título do produto
- **Output**: Descrição do produto

O formato seguirá o padrão de chat do Llama 3, usando tags especiais para delimitar o início e fim das respostas.

In [ ]:
def format_prompt(title, content):
    """
    Formata um exemplo de treinamento no padrão do Llama 3
    
    Args:
        title: Título do produto
        content: Descrição do produto
    
    Returns:
        String formatada para treinamento
    """
    
    # Template de prompt para o modelo
    prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Você é um assistente especializado em produtos da Amazon. Sua tarefa é gerar descrições detalhadas e precisas de produtos baseadas apenas no título fornecido. As descrições devem ser informativas, concisas e atrativas para potenciais compradores.<|eot_id|><|start_header_id|>user<|end_header_id|>

Gere uma descrição para o seguinte produto: {title}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{content}<|eot_id|><|end_of_text|>"""
    
    return prompt_template.format(title=title, content=content)

def prepare_training_data(data, train_size=None):
    """
    Prepara os dados para treinamento formatando cada exemplo
    
    Args:
        data: Lista de dicionários com title e content
        train_size: Número máximo de exemplos para treinamento
    
    Returns:
        Lista de strings formatadas para treinamento
    """
    
    print(f"🔄 Preparando dados para treinamento...")
    
    if train_size:
        data = data[:train_size]
    
    formatted_data = []
    
    for i, item in enumerate(data):
        formatted_prompt = format_prompt(item['title'], item['content'])
        formatted_data.append(formatted_prompt)
        
        # Progress update
        if (i + 1) % 500 == 0:
            print(f"  Formatados {i + 1} exemplos...")
    
    print(f"✅ Preparação concluída! Total: {len(formatted_data)} exemplos formatados")
    return formatted_data

# Testa a formatação com um exemplo
if sample_data:
    print("🧪 TESTE DE FORMATAÇÃO")
    print("=" * 60)
    
    # Pega um exemplo para demonstrar a formatação
    test_example = sample_data[0]
    formatted_example = format_prompt(test_example['title'], test_example['content'])
    
    print("📝 Exemplo original:")
    print(f"  Título: {test_example['title']}")
    print(f"  Conteúdo: {test_example['content']}")
    
    print("\n🎯 Exemplo formatado para treinamento:")
    print("-" * 60)
    print(formatted_example)
    print("-" * 60)
    
else:
    print("❌ Nenhum dado disponível para teste.")

### 3.3 Divisão em Conjuntos de Treino e Teste

Agora que temos os dados limpos, vamos dividi-los em conjuntos de treinamento e teste.

### 3.2 Limpeza e Pré-processamento dos Dados

Antes de prosseguir com o treinamento, é essencial limpar os dados removendo duplicatas, valores nulos, textos muito curtos ou muito longos, e outros problemas de qualidade.

In [ ]:
def clean_amazon_data(data, min_title_length=3, max_title_length=200, 
                      min_content_length=5, max_content_length=1000):
    """
    Limpa e pré-processa os dados do Amazon dataset
    
    Args:
        data: Lista de dicionários com title e content
        min_title_length: Comprimento mínimo do título
        max_title_length: Comprimento máximo do título
        min_content_length: Comprimento mínimo do conteúdo
        max_content_length: Comprimento máximo do conteúdo
    
    Returns:
        Lista de dados limpos e estatísticas da limpeza
    """
    
    print("🧹 INICIANDO LIMPEZA DOS DADOS")
    print("=" * 50)
    
    # Estatísticas iniciais
    initial_count = len(data)
    print(f"📊 Dados iniciais: {initial_count} amostras")
    
    # 1. Remove valores nulos ou vazios
    print("\n🔍 1. Removendo valores nulos ou vazios...")
    data = [item for item in data if item.get('title') and item.get('content')]
    after_null_removal = len(data)
    removed_null = initial_count - after_null_removal
    print(f"   Removidas: {removed_null} amostras")
    print(f"   Restantes: {after_null_removal} amostras")
    
    # 2. Remove whitespace extra e normaliza
    print("\n✂️ 2. Normalizando espaços em branco...")
    for item in data:
        item['title'] = ' '.join(item['title'].split())  # Remove espaços extras
        item['content'] = ' '.join(item['content'].split())
    
    # 3. Remove duplicatas baseadas no título
    print("\n🔄 3. Removendo duplicatas...")
    seen_titles = set()
    unique_data = []
    duplicates_removed = 0
    
    for item in data:
        title_lower = item['title'].lower()
        if title_lower not in seen_titles:
            seen_titles.add(title_lower)
            unique_data.append(item)
        else:
            duplicates_removed += 1
    
    data = unique_data
    print(f"   Duplicatas removidas: {duplicates_removed}")
    print(f"   Restantes: {len(data)} amostras")
    
    # 4. Filtra por comprimento do título
    print(f"\n📏 4. Filtrando títulos (min: {min_title_length}, max: {max_title_length} chars)...")
    before_title_filter = len(data)
    data = [item for item in data if min_title_length <= len(item['title']) <= max_title_length]
    removed_title = before_title_filter - len(data)
    print(f"   Removidas: {removed_title} amostras")
    print(f"   Restantes: {len(data)} amostras")
    
    # 5. Filtra por comprimento do conteúdo
    print(f"\n📄 5. Filtrando conteúdo (min: {min_content_length}, max: {max_content_length} chars)...")
    before_content_filter = len(data)
    data = [item for item in data if min_content_length <= len(item['content']) <= max_content_length]
    removed_content = before_content_filter - len(data)
    print(f"   Removidas: {removed_content} amostras")
    print(f"   Restantes: {len(data)} amostras")
    
    # 6. Remove caracteres especiais problemáticos
    print("\n🔧 6. Limpando caracteres especiais...")
    import re
    
    for item in data:\n        # Remove caracteres de controle e caracteres não imprimíveis
        item['title'] = re.sub(r'[\\x00-\\x1f\\x7f-\\x9f]', '', item['title'])
        item['content'] = re.sub(r'[\\x00-\\x1f\\x7f-\\x9f]', '', item['content'])
        
        # Remove múltiplas quebras de linha
        item['content'] = re.sub(r'\\n+', ' ', item['content'])
        
        # Remove espaços extras novamente
        item['title'] = ' '.join(item['title'].split())
        item['content'] = ' '.join(item['content'].split())
    
    # 7. Verificação final de qualidade
    print("\n✅ 7. Verificação final de qualidade...")
    final_data = []
    removed_final = 0
    
    for item in data:
        # Verifica se ainda tem conteúdo válido
        if (item['title'].strip() and item['content'].strip() and 
            len(item['title'].strip()) >= min_title_length and
            len(item['content'].strip()) >= min_content_length):
            final_data.append(item)
        else:
            removed_final += 1
    
    data = final_data
    print(f"   Removidas na verificação final: {removed_final}")
    print(f"   Total final: {len(data)} amostras")
    
    # Estatísticas de limpeza
    total_removed = initial_count - len(data)
    retention_rate = (len(data) / initial_count) * 100
    
    print(f"\n📈 RESUMO DA LIMPEZA:")
    print(f"   Dados iniciais: {initial_count}")
    print(f"   Dados finais: {len(data)}")
    print(f"   Total removido: {total_removed} ({(total_removed/initial_count)*100:.1f}%)")
    print(f"   Taxa de retenção: {retention_rate:.1f}%")
    
    # Estatísticas dos dados limpos
    if data:
        title_lengths = [len(item['title']) for item in data]
        content_lengths = [len(item['content']) for item in data]
        
        print(f"\n📊 ESTATÍSTICAS DOS DADOS LIMPOS:")
        print(f"   Títulos - Mín: {min(title_lengths)}, Máx: {max(title_lengths)}, Média: {np.mean(title_lengths):.1f}")
        print(f"   Conteúdo - Mín: {min(content_lengths)}, Máx: {max(content_lengths)}, Média: {np.mean(content_lengths):.1f}")
    
    return data

# Aplica a limpeza nos dados carregados
if raw_data:
    print("🔄 Aplicando limpeza nos dados carregados...")
    cleaned_data = clean_amazon_data(raw_data.copy())
else:
    print("❌ Nenhum dado disponível para limpeza.")
    cleaned_data = []

In [ ]:
# Comparação visual: Antes vs Depois da limpeza
if raw_data and cleaned_data:
    print("📊 COMPARAÇÃO VISUAL: ANTES vs DEPOIS DA LIMPEZA")
    print("=" * 60)
    
    # Cria visualizações comparativas
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Comparação: Dados Originais vs Dados Limpos', fontsize=16, fontweight='bold')
    
    # Dados originais
    orig_title_lengths = [len(item['title']) for item in raw_data]
    orig_content_lengths = [len(item['content']) for item in raw_data]
    
    # Dados limpos
    clean_title_lengths = [len(item['title']) for item in cleaned_data]
    clean_content_lengths = [len(item['content']) for item in cleaned_data]
    
    # Gráfico 1: Contagem de amostras
    categories = ['Dados Originais', 'Dados Limpos']
    counts = [len(raw_data), len(cleaned_data)]
    colors = ['lightcoral', 'lightgreen']
    
    axes[0,0].bar(categories, counts, color=colors, alpha=0.7, edgecolor='black')
    axes[0,0].set_title('Número Total de Amostras')
    axes[0,0].set_ylabel('Quantidade')
    for i, v in enumerate(counts):
        axes[0,0].text(i, v + max(counts)*0.01, str(v), ha='center', fontweight='bold')
    
    # Gráfico 2: Distribuição títulos - Originais
    axes[0,1].hist(orig_title_lengths, bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[0,1].set_title('Títulos - Dados Originais')
    axes[0,1].set_xlabel('Comprimento (caracteres)')
    axes[0,1].set_ylabel('Frequência')
    axes[0,1].grid(True, alpha=0.3)
    
    # Gráfico 3: Distribuição títulos - Limpos
    axes[0,2].hist(clean_title_lengths, bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
    axes[0,2].set_title('Títulos - Dados Limpos')
    axes[0,2].set_xlabel('Comprimento (caracteres)')
    axes[0,2].set_ylabel('Frequência')
    axes[0,2].grid(True, alpha=0.3)
    
    # Gráfico 4: Distribuição conteúdo - Originais
    axes[1,0].hist(orig_content_lengths, bins=30, alpha=0.7, color='lightcoral', edgecolor='black')
    axes[1,0].set_title('Conteúdo - Dados Originais')
    axes[1,0].set_xlabel('Comprimento (caracteres)')
    axes[1,0].set_ylabel('Frequência')
    axes[1,0].grid(True, alpha=0.3)
    
    # Gráfico 5: Distribuição conteúdo - Limpos
    axes[1,1].hist(clean_content_lengths, bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
    axes[1,1].set_title('Conteúdo - Dados Limpos')
    axes[1,1].set_xlabel('Comprimento (caracteres)')
    axes[1,1].set_ylabel('Frequência')
    axes[1,1].grid(True, alpha=0.3)
    
    # Gráfico 6: Boxplot comparativo
    data_to_plot = [orig_title_lengths, clean_title_lengths, orig_content_lengths, clean_content_lengths]
    labels = ['Títulos\\nOriginais', 'Títulos\\nLimpos', 'Conteúdo\\nOriginais', 'Conteúdo\\nLimpos']
    colors_box = ['lightcoral', 'lightgreen', 'lightcoral', 'lightgreen']
    
    bp = axes[1,2].boxplot(data_to_plot, labels=labels, patch_artist=True)
    for patch, color in zip(bp['boxes'], colors_box):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    axes[1,2].set_title('Comparação de Distribuições')
    axes[1,2].set_ylabel('Comprimento (caracteres)')
    axes[1,2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Estatísticas comparativas em tabela
    print("\\n📋 ESTATÍSTICAS COMPARATIVAS:")
    print("-" * 80)
    print(f"{'Métrica':<25} {'Originais':<15} {'Limpos':<15} {'Variação':<15}")
    print("-" * 80)
    
    # Total de amostras
    original_count = len(raw_data)
    clean_count = len(cleaned_data)
    variation = ((clean_count - original_count) / original_count) * 100
    print(f"{'Total de amostras':<25} {original_count:<15} {clean_count:<15} {variation:+.1f}%")
    
    # Estatísticas de títulos
    orig_title_mean = np.mean(orig_title_lengths)
    clean_title_mean = np.mean(clean_title_lengths)
    title_variation = ((clean_title_mean - orig_title_mean) / orig_title_mean) * 100
    print(f"{'Título médio (chars)':<25} {orig_title_mean:<15.1f} {clean_title_mean:<15.1f} {title_variation:+.1f}%")
    
    # Estatísticas de conteúdo
    orig_content_mean = np.mean(orig_content_lengths)
    clean_content_mean = np.mean(clean_content_lengths)
    content_variation = ((clean_content_mean - orig_content_mean) / orig_content_mean) * 100
    print(f"{'Conteúdo médio (chars)':<25} {orig_content_mean:<15.1f} {clean_content_mean:<15.1f} {content_variation:+.1f}%")
    
    print("-" * 80)
    
else:
    print("❌ Dados para comparação não disponíveis.")

In [ ]:
# Exemplos de dados problemáticos que foram removidos
if raw_data and cleaned_data:
    print("🔍 EXEMPLOS DE DADOS PROBLEMÁTICOS REMOVIDOS")
    print("=" * 60)
    
    # Identifica dados que foram removidos
    original_titles = {item['title'] for item in raw_data}
    clean_titles = {item['title'] for item in cleaned_data}
    removed_titles = original_titles - clean_titles
    
    if removed_titles:
        print(f"📊 Total de títulos únicos removidos: {len(removed_titles)}")
        
        # Encontra exemplos específicos dos dados removidos
        removed_examples = []
        for item in raw_data:
            if item['title'] in removed_titles:
                removed_examples.append(item)
                if len(removed_examples) >= 5:  # Mostra até 5 exemplos
                    break
        
        print("\\n❌ EXEMPLOS DE DADOS REMOVIDOS:")
        print("-" * 60)
        
        for i, example in enumerate(removed_examples, 1):
            print(f"\\n{i}. EXEMPLO PROBLEMÁTICO:")
            print(f"   Título: '{example['title']}'")
            print(f"   Conteúdo: '{example['content']}'")
            
            # Identifica o problema
            problems = []
            if len(example['title']) < 3:
                problems.append(f"título muito curto ({len(example['title'])} chars)")
            if len(example['title']) > 200:
                problems.append(f"título muito longo ({len(example['title'])} chars)")
            if len(example['content']) < 5:
                problems.append(f"conteúdo muito curto ({len(example['content'])} chars)")
            if len(example['content']) > 1000:
                problems.append(f"conteúdo muito longo ({len(example['content'])} chars)")
            if not example['title'].strip():
                problems.append("título vazio")
            if not example['content'].strip():
                problems.append("conteúdo vazio")
            
            print(f"   ⚠️ Problema(s): {', '.join(problems) if problems else 'duplicata'}")
    
    else:
        print("✅ Nenhum dado foi removido - todos estavam dentro dos critérios!")
    
    # Mostra exemplos de dados que PERMANECERAM após limpeza
    print("\\n\\n✅ EXEMPLOS DE DADOS LIMPOS (QUE PERMANECERAM):")
    print("=" * 60)
    
    for i in range(min(3, len(cleaned_data))):
        example = cleaned_data[i]
        print(f"\\n{i+1}. EXEMPLO LIMPO:")
        print(f"   Título: '{example['title']}'")
        print(f"   Conteúdo: '{example['content']}'")
        print(f"   📏 Título: {len(example['title'])} chars, Conteúdo: {len(example['content'])} chars")
        print(f"   ✅ Status: Dados válidos e dentro dos parâmetros")

else:
    print("❌ Dados para análise de exemplos não disponíveis.")

In [ ]:
# Divisão dos dados limpos em treino e teste
if cleaned_data:
    print("? DIVIDINDO DADOS LIMPOS EM TREINO E TESTE")
    print("=" * 50)
    
    print(f"📊 Total de dados limpos disponíveis: {len(cleaned_data)}")
    
    # Configuração da divisão
    test_size = min(CONFIG['test_size'], len(cleaned_data) // 10)  # Máximo 10% para teste
    train_size = len(cleaned_data) - test_size
    
    # Embaralha os dados antes de dividir
    import random
    random.shuffle(cleaned_data)
    
    # Divisão dos dados
    train_data = cleaned_data[:train_size]
    test_data = cleaned_data[train_size:train_size + test_size]
    
    print(f"  📚 Dados de treinamento: {len(train_data)}")
    print(f"  🧪 Dados de teste: {len(test_data)}")
    print(f"  📊 Proporção treino/teste: {len(train_data)/len(test_data):.1f}")
    
    # Estatísticas dos conjuntos
    train_title_lengths = [len(item['title']) for item in train_data]
    train_content_lengths = [len(item['content']) for item in train_data]
    
    test_title_lengths = [len(item['title']) for item in test_data]
    test_content_lengths = [len(item['content']) for item in test_data]
    
    print(f"\n📏 ESTATÍSTICAS DO CONJUNTO DE TREINAMENTO:")
    print(f"  Títulos - Média: {np.mean(train_title_lengths):.1f} caracteres")
    print(f"  Conteúdo - Média: {np.mean(train_content_lengths):.1f} caracteres")
    
    print(f"\n📏 ESTATÍSTICAS DO CONJUNTO DE TESTE:")
    print(f"  Títulos - Média: {np.mean(test_title_lengths):.1f} caracteres")
    print(f"  Conteúdo - Média: {np.mean(test_content_lengths):.1f} caracteres")
    
    print(f"\n✅ Divisão concluída com sucesso!")
    print(f"🎯 Dados prontos para formatação e treinamento")
    
else:
    print("❌ Nenhum dado limpo disponível para divisão.")
    train_data = []
    test_data = []

### 3.4 Criação do Dataset no Formato Hugging Face

Vamos criar datasets no formato esperado pela biblioteca Hugging Face para facilitar o treinamento.

In [ ]:
# Prepara os dados formatados para treinamento
if 'train_data' in locals() and train_data:
    print("🔧 CRIANDO DATASETS FORMATADOS")
    print("=" * 50)
    
    # Formata os dados de treinamento
    formatted_train_data = prepare_training_data(train_data)
    
    # Cria o dataset de treinamento no formato Hugging Face
    train_dataset_dict = {
        'text': formatted_train_data
    }
    
    train_dataset = Dataset.from_dict(train_dataset_dict)
    
    print(f"\n✅ Dataset de treinamento criado:")
    print(f"   Número de exemplos: {len(train_dataset)}")
    print(f"   Colunas: {train_dataset.column_names}")
    
    # Salva algumas amostras dos dados de teste para avaliação posterior
    test_samples = test_data[:10]  # Primeiras 10 amostras para teste
    
    print(f"\n📋 Amostras de teste separadas: {len(test_samples)}")
    
    # Mostra estatísticas do dataset final
    text_lengths = [len(text) for text in formatted_train_data]
    print(f"\n📊 ESTATÍSTICAS DO DATASET FORMATADO:")
    print(f"   Comprimento médio do texto: {np.mean(text_lengths):.0f} caracteres")
    print(f"   Comprimento mínimo: {min(text_lengths)} caracteres")
    print(f"   Comprimento máximo: {max(text_lengths)} caracteres")
    
    # Verifica se os textos não são muito longos para o modelo
    max_length = CONFIG['max_seq_length']
    long_texts = [t for t in text_lengths if t > max_length * 4]  # Aproximadamente 4 chars por token
    
    if long_texts:
        print(f"   ⚠️ Textos muito longos: {len(long_texts)} ({len(long_texts)/len(text_lengths)*100:.1f}%)")
    else:
        print(f"   ✅ Todos os textos estão dentro do limite esperado")
    
else:
    print("❌ Dados de treinamento não disponíveis.")

In [ ]:
# Mostra exemplos do dataset formatado
if 'train_dataset' in locals() and train_dataset:
    print("👀 VISUALIZAÇÃO DOS DADOS FORMATADOS")
    print("=" * 60)
    
    # Mostra 2 exemplos completos
    for i in range(min(2, len(train_dataset))):
        print(f"\n📝 EXEMPLO {i+1}:")
        print("-" * 40)
        
        # Pega o texto formatado
        formatted_text = train_dataset[i]['text']
        
        # Extrai partes específicas para visualização
        lines = formatted_text.split('\n')
        
        # Encontra o título (depois de "Gere uma descrição para o seguinte produto:")
        title_line = None
        content_start = None
        
        for j, line in enumerate(lines):
            if "Gere uma descrição para o seguinte produto:" in line:
                if j + 1 < len(lines):
                    title_line = lines[j + 1].strip()
            elif "<|start_header_id|>assistant<|end_header_id|>" in line:
                content_start = j + 1
                break
        
        if title_line:
            print(f"🏷️  Título: {title_line}")
        
        if content_start and content_start < len(lines):
            # Pega o conteúdo (até encontrar <|eot_id|>)
            content_lines = []
            for k in range(content_start, len(lines)):
                if "<|eot_id|>" in lines[k]:
                    break
                if lines[k].strip():
                    content_lines.append(lines[k].strip())
            
            content = " ".join(content_lines)
            if content:
                print(f"📄 Descrição: {content}")
        
        print(f"📏 Comprimento total: {len(formatted_text)} caracteres")
    
    print(f"\n✅ Dataset pronto para treinamento!")
    print(f"📊 Resumo final: {len(train_dataset)} exemplos formatados")
    
else:
    print("❌ Dataset formatado não disponível.")